# Analyse processed Tweets

Problems:

- Commas within text make it difficult to read files > when preprocessing, add a special delimiter to mark the beginning and the start of the Tweet content

In [76]:
# Useful starting lines
%matplotlib inline
import copy
import numpy as np 
#import matplotlib as mpl
#import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd
import re
import glob
import ast
from datetime import datetime
from collections import Counter
import itertools
import nltk
nltk.download("stopwords")
from nltk import bigrams 
import string
import locale
import reverse_geocoder as rg
locale.setlocale(locale.LC_TIME, "en_US.utf8")
%load_ext autoreload

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/drosoneuro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Load and combine Data

In [174]:
#idea read whole line >  then parse
def load_csv_data(data_path, sub_sample=False):
    """Loads data and transforms it into a pandas dataframe"""
    
    #extracts tweet ID and timestamp
    #tw1 = pd.read_csv(data_path,sep=",",usecols=[0,1],encoding = "ISO-8859-1",header=None)
    
    #extracts tweet content
    #tw2 = pd.read_csv(data_path,sep="[0-9],|,\"\[",usecols=[2],encoding = "ISO-8859-1",header=None,engine="python")
    
    #extract coordinates & location tag
    #tw3 = pd.read_csv(data_path,sep=",\"\[|\]\",",usecols=[1,2],encoding = "ISO-8859-1",header=None,engine="python")
    
    #merge everything
    #tw = pd.concat((tw1,tw2,tw3),axis=1)
    
    #extract complete string per row
    tw = pd.read_csv(data_path,sep="NotADelimiter",usecols=[0,1],encoding = "ISO-8859-1",header=None,engine="python")  
    
    #extract tweet ID and timestamp
    s = tw[0].apply(lambda x: x.split(","))
    tw["id"] = s.apply(lambda x: x[0])
    tw["time"] = s.apply(lambda x: x[1] if len(x) > 1 else None)
    
    #extract tweet content
    s = tw[0].apply(lambda x: re.split("[0-9],|,\"\[",x))
    tw["text"] = s.apply(lambda x: x[2] if len(x) > 2 else None)
    
    #extract coordinates & location tag
    s = tw[0].apply(lambda x: re.split(",\"\[|\]\",",x))
    tw["coord"] = s.apply(lambda x: x[1] if len(x)> 1 else None)
    tw["loc_tag"] = s.apply(lambda x: x[2] if len(x) > 2 else None)
    #tw3 = pd.read_csv(data_path,sep=",\"\[|\]\",",usecols=[1,2],encoding = "ISO-8859-1",header=None,engine="python")
    
    
    # sub-sample
    if sub_sample:
        tw = tw[::50]
    return tw

In [175]:
"""Read all files in Data directory, load & parse them and concatenate resulting DataFrames"""
files = glob.glob('./Data/*.csv')
tw = pd.DataFrame
first = True
for data_path in files:
    tw_temp = load_csv_data(data_path=data_path, sub_sample=True)
    if first:
        tw = tw_temp
        first = False
    else:
        tw = pd.concat((tw,tw_temp),axis=0,ignore_index=False)
tw.index = range(0,np.shape(tw)[0])

In [176]:
tw

,0,id,time,text,coord,loc_tag
0,"313741620505890817,Mon Mar 18 20:00:06 +0000 2...",313741620505890817,Mon Mar 18 20:00:06 +0000 2013,got outstanding student of the semester :) sup...,"-74.2041177, 40.9002544",coordinates
1,"313741620602339329,Mon Mar 18 20:00:06 +0000 2...",313741620602339329,Mon Mar 18 20:00:06 +0000 2013,"""Rachael Harris Sighting: Culver City, Califor...","-118.387, 34.0188",coordinates
2,"313741620707196928,Mon Mar 18 20:00:06 +0000 2...",313741620707196928,Mon Mar 18 20:00:06 +0000 2013,@simplycleanme ain't that the truth,"-121.62528736, 39.11648576",coordinates
3,"313741619650240513,Mon Mar 18 20:00:06 +0000 2...",313741619650240513,Mon Mar 18 20:00:06 +0000 2013,My toy for the week is toooo niceeee http://t....,"-81.61230074, 28.10675658",coordinates
4,"313741620338114560,Mon Mar 18 20:00:05 +0000 2...",313741620338114560,Mon Mar 18 20:00:05 +0000 2013,Im about to win this march madness tourney. #L...,"-104.98052025, 39.65046024",coordinates
5,"313741621386702848,Mon Mar 18 20:00:06 +0000 2...",313741621386702848,Mon Mar 18 20:00:06 +0000 2013,"""Mick Lee, Managing Director, Head of Global T...","-73.9858173, 40.7585478",coordinates
6,"313741621357322241,Mon Mar 18 20:00:06 +0000 2...",313741621357322241,Mon Mar 18 20:00:06 +0000 2013,Sat in an English class at SSU today.,"-82.40254889, 32.20054948",coordinates
7,"313741621340565504,Mon Mar 18 20:00:06 +0000 2...",313741621340565504,Mon Mar 18 20:00:06 +0000 2013,Heading home from the YMCA. Good shoot around,"-82.07645198, 41.37846295",coordinates
8,"313741619696394241,Mon Mar 18 20:00:06 +0000 2...",313741619696394241,Mon Mar 18 20:00:06 +0000 2013,Now that looks good Goat Cheese Arugula Pizza ...,"-71.04177291, 42.47275057",coordinates
9,"313741621697052673,Mon Mar 18 20:00:06 +0000 2...",313741621697052673,Mon Mar 18 20:00:06 +0000 2013,@CourtNeeCheske you're no bother:) i sit on my...,"-88.0285948, 42.9729355",coordinates


In [177]:
tw["coord"] = tw["coord"].apply(lambda x: re.findall(r"[-+]?\d*\.\d+|\d+",x) if isinstance(x,str) else None)
tw["coord"] = tw["coord"].apply(lambda x: [float(i) for i in x] if isinstance(x,list) else None)

In [178]:
"""assign a single coordinate to rows with location tag "place"""
indx = tw["loc_tag"]=="place"
tw_place = tw[indx]["coord"]
#tw_place = tw_place.apply(lambda x: x.replace("\"",""))
#tw_place =tw_place.apply(lambda x: ast.literal_eval(x))

#calculate the mean of the coordinates given; 
#because the coordinates are very close to each other, we can treat the surface as flat
def calc_mean(x):
    if x == None:
        return(None)
    else:
        lons = [x[i] for i in range(0,len(x),2)]
        lats = [x[i] for i in range(1,len(x),2)]
    lon = np.mean(lons)
    lat = np.mean(lats)
    return([lon,lat])

tw_place = tw_place.apply(calc_mean)
tw.loc[indx,"coord"] = tw_place

In [179]:
#convert time strings to datetime formate
tw["time"] = tw["time"].apply(lambda x: re.sub("\+0000\s","",str(x)) if re.search("\+0000\s",str(x)) else None)
tw["time"] = tw["time"].apply(lambda x: datetime.strptime(x,"%a %b %d %H:%M:%S %Y") if isinstance(x,str) else None)

In [181]:
#convert id-string to int
tw["id"] = tw["id"].apply(lambda x: float(x) if re.search("^[0-9]{18}$",x) else None)

In [182]:
#remove first colum with full row
del tw[0]

#remove all rows which contain None in any row
indx = tw.apply(lambda x: False if x.isnull().any() else True,axis=1)
tw = tw[indx]
print("Removed a total of "+ str(sum(~indx)) + " rows")

Removed a total of 76784 rows


In [183]:
#Tokenize & Count tokens in tweet text
#from https://marcobonzanini.com/2015/03/09/mining-twitter-data-with-python-part-2/
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)

def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False,stopwords_rm=False,**kwargs):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    if stopwords_rm:
        tokens = [token for token in tokens if token not in kwargs["stop_words"]]
    return tokens

In [184]:
tw["tokens"] = tw["text"].apply(preprocess,lowercase=True)

/home/drosoneuro/anaconda3/envs/python_ml/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [185]:
#Count occurrence of each word
counts = Counter(itertools.chain.from_iterable(tw["tokens"]))

In [186]:
counts

Counter({'http://t.co/pjapprzd7x': 1,
         '@lucykarnold': 1,
         'deed': 43,
         '@abbyoz27': 10,
         '@emily_goodman6': 2,
         '@maggie_farley': 1,
         '@djmarkfarina': 1,
         'http://t.co/hckbr7re': 1,
         '@ezernack_pookie': 2,
         'bachatu': 1,
         '@bilalmirzaisraw': 1,
         'http://t.co/g0cd8fkrsa': 1,
         'skanking': 1,
         '@christina_marii': 1,
         'flstse': 1,
         'svng': 1,
         'http://t.co/9azgr8no': 1,
         'http://t.co/2nuxktwx': 1,
         'citei': 2,
         'http://t.co/m52lfxin': 1,
         '@tristentaylorr': 1,
         'http://t.co/496xbswp': 1,
         'http://t.co/fk4tzu4f': 1,
         'http://t.co/xpm07eomw': 1,
         '#awesomebymyself': 1,
         '@tweetrajouhari': 1,
         "jackee's": 1,
         'catepillar': 1,
         'http://t.co/1q4cexeu': 1,
         '#imhopeless': 1,
         'panooooo': 1,
         '@aroad_jenny': 1,
         'appritiate': 1,
         '@sara

In [188]:
#remove punctuation and stopwords
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via']
tw["tokens_stop"] = tw["text"].apply(preprocess,lowercase=True,
                              stopwords_rm = True, stop_words=stop)

/home/drosoneuro/anaconda3/envs/python_ml/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [189]:
#create bigrams
tw["bigrams"] = tw["tokens_stop"].apply(bigrams)

/home/drosoneuro/anaconda3/envs/python_ml/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [190]:
tw

,id,time,text,coord,loc_tag,tokens,tokens_stop,bigrams
0,3.137416e+17,2013-03-18 20:00:06,got outstanding student of the semester :) sup...,"[-74.2041177, 40.9002544]",coordinates,"[got, outstanding, student, of, the, semester,...","[got, outstanding, student, semester, :), supe...",<generator object bigrams at 0x7f7e0c5da7d8>
1,3.137416e+17,2013-03-18 20:00:06,"""Rachael Harris Sighting: Culver City, Califor...","[-118.387, 34.0188]",coordinates,"["", rachael, harris, sighting, :, culver, city...","[rachael, harris, sighting, culver, city, cali...",<generator object bigrams at 0x7f7e0adb2a98>
2,3.137416e+17,2013-03-18 20:00:06,@simplycleanme ain't that the truth,"[-121.62528736, 39.11648576]",coordinates,"[@simplycleanme, ain't, that, the, truth]","[@simplycleanme, ain't, truth]",<generator object bigrams at 0x7f7e0adb2938>
3,3.137416e+17,2013-03-18 20:00:06,My toy for the week is toooo niceeee http://t....,"[-81.61230074, 28.10675658]",coordinates,"[my, toy, for, the, week, is, toooo, niceeee, ...","[toy, week, toooo, niceeee, http://t.co/rbnjte...",<generator object bigrams at 0x7f7e0c2bf048>
4,3.137416e+17,2013-03-18 20:00:05,Im about to win this march madness tourney. #L...,"[-104.98052025, 39.65046024]",coordinates,"[im, about, to, win, this, march, madness, tou...","[im, win, march, madness, tourney, #letsgooo]",<generator object bigrams at 0x7f7e0c2bfe60>
5,3.137416e+17,2013-03-18 20:00:06,"""Mick Lee, Managing Director, Head of Global T...","[-73.9858173, 40.7585478]",coordinates,"["", mick, lee, ,, managing, director, ,, head,...","[mick, lee, managing, director, head, global, ...",<generator object bigrams at 0x7f7e0c2bf6d0>
6,3.137416e+17,2013-03-18 20:00:06,Sat in an English class at SSU today.,"[-82.40254889, 32.20054948]",coordinates,"[sat, in, an, english, class, at, ssu, today, .]","[sat, english, class, ssu, today]",<generator object bigrams at 0x7f7e0c2bff68>
7,3.137416e+17,2013-03-18 20:00:06,Heading home from the YMCA. Good shoot around,"[-82.07645198, 41.37846295]",coordinates,"[heading, home, from, the, ymca, ., good, shoo...","[heading, home, ymca, good, shoot, around]",<generator object bigrams at 0x7f7e0c2bfc50>
8,3.137416e+17,2013-03-18 20:00:06,Now that looks good Goat Cheese Arugula Pizza ...,"[-71.04177291, 42.47275057]",coordinates,"[now, that, looks, good, goat, cheese, arugula...","[looks, good, goat, cheese, arugula, pizza, ht...",<generator object bigrams at 0x7f7de4a4ad58>
9,3.137416e+17,2013-03-18 20:00:06,@CourtNeeCheske you're no bother:) i sit on my...,"[-88.0285948, 42.9729355]",coordinates,"[@courtneecheske, you're, no, bother, :), i, s...","[@courtneecheske, you're, bother, :), sit, pho...",<generator object bigrams at 0x7f7e1445fbf8>


In [199]:
#switch lat & lon position to work with reverse geocoder
tw["coord"] = tw["coord"].apply(lambda x: [x[1],x[0]] if len(x)>1 else None)

/home/drosoneuro/anaconda3/envs/python_ml/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [208]:
tw = tw[[i is not None for i in tw["coord"]]]

In [209]:
#get state location for each tweet
def state_lookup(x):
    coords =[(i,j) for i,j in x.values.flatten()]
    loc_info = rg.search(coords)
    states = [i["admin1"] for i in loc_info]
    return(states)
tw["state"] = state_lookup(tw["coord"])

/home/drosoneuro/anaconda3/envs/python_ml/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [210]:
tw

,id,time,text,coord,loc_tag,tokens,tokens_stop,bigrams,state
0,3.137416e+17,2013-03-18 20:00:06,got outstanding student of the semester :) sup...,"[40.9002544, -74.2041177]",coordinates,"[got, outstanding, student, of, the, semester,...","[got, outstanding, student, semester, :), supe...",<generator object bigrams at 0x7f7e0c5da7d8>,New Jersey
1,3.137416e+17,2013-03-18 20:00:06,"""Rachael Harris Sighting: Culver City, Califor...","[34.0188, -118.387]",coordinates,"["", rachael, harris, sighting, :, culver, city...","[rachael, harris, sighting, culver, city, cali...",<generator object bigrams at 0x7f7e0adb2a98>,California
2,3.137416e+17,2013-03-18 20:00:06,@simplycleanme ain't that the truth,"[39.11648576, -121.62528736]",coordinates,"[@simplycleanme, ain't, that, the, truth]","[@simplycleanme, ain't, truth]",<generator object bigrams at 0x7f7e0adb2938>,California
3,3.137416e+17,2013-03-18 20:00:06,My toy for the week is toooo niceeee http://t....,"[28.10675658, -81.61230074]",coordinates,"[my, toy, for, the, week, is, toooo, niceeee, ...","[toy, week, toooo, niceeee, http://t.co/rbnjte...",<generator object bigrams at 0x7f7e0c2bf048>,Florida
4,3.137416e+17,2013-03-18 20:00:05,Im about to win this march madness tourney. #L...,"[39.65046024, -104.98052025]",coordinates,"[im, about, to, win, this, march, madness, tou...","[im, win, march, madness, tourney, #letsgooo]",<generator object bigrams at 0x7f7e0c2bfe60>,Colorado
5,3.137416e+17,2013-03-18 20:00:06,"""Mick Lee, Managing Director, Head of Global T...","[40.7585478, -73.9858173]",coordinates,"["", mick, lee, ,, managing, director, ,, head,...","[mick, lee, managing, director, head, global, ...",<generator object bigrams at 0x7f7e0c2bf6d0>,New York
6,3.137416e+17,2013-03-18 20:00:06,Sat in an English class at SSU today.,"[32.20054948, -82.40254889]",coordinates,"[sat, in, an, english, class, at, ssu, today, .]","[sat, english, class, ssu, today]",<generator object bigrams at 0x7f7e0c2bff68>,Georgia
7,3.137416e+17,2013-03-18 20:00:06,Heading home from the YMCA. Good shoot around,"[41.37846295, -82.07645198]",coordinates,"[heading, home, from, the, ymca, ., good, shoo...","[heading, home, ymca, good, shoot, around]",<generator object bigrams at 0x7f7e0c2bfc50>,Ohio
8,3.137416e+17,2013-03-18 20:00:06,Now that looks good Goat Cheese Arugula Pizza ...,"[42.47275057, -71.04177291]",coordinates,"[now, that, looks, good, goat, cheese, arugula...","[looks, good, goat, cheese, arugula, pizza, ht...",<generator object bigrams at 0x7f7de4a4ad58>,Massachusetts
9,3.137416e+17,2013-03-18 20:00:06,@CourtNeeCheske you're no bother:) i sit on my...,"[42.9729355, -88.0285948]",coordinates,"[@courtneecheske, you're, no, bother, :), i, s...","[@courtneecheske, you're, bother, :), sit, pho...",<generator object bigrams at 0x7f7e1445fbf8>,Wisconsin
